In [1]:
import numpy as np
import pandas as pd

In [2]:
df_ = pd.read_table('ICEWS Coded Event Data/events.2014.20160121105408.tab')

In [3]:
df_.head()

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,20718170,2014-01-01,Police (Australia),"Police,Government",Australia,"Arrest, detain, or charge with legal action",173,-5,Men (Australia),"Social,General Population / Civilian / Social",Australia,32493690,2,Daily Telegraph,Surfers Paradise,Gold Coast,State of Queensland,Australia,-28.00270,153.4300
1,20718171,2014-01-01,Police (Australia),"Police,Government",Australia,"Arrest, detain, or charge with legal action",173,-5,Children (Australia),"Social,General Population / Civilian / Social",Australia,32493693,1,Daily Telegraph,Maroubra,Randwick,State of New South Wales,Australia,-33.95000,151.2330
2,20718172,2014-01-01,Government Official (Democratic Republic of Co...,Government,Democratic Republic of Congo,Make statement,10,0,Attacker (Democratic Republic of Congo),"Criminals / Gangs,Dissident",Democratic Republic of Congo,32495112,3,The Australian,Kinshasa,NaN,Kinshasa City,Democratic Republic of Congo,-4.32142,15.3081
3,20718174,2014-01-01,Military (South Sudan),"Military,Government",South Sudan,Use conventional military force,190,-10,Armed Rebel (South Sudan),"Rebel,Dissident",South Sudan,32495113,1,The Australian,Juba,NaN,Central Equatoria State,South Sudan,4.85165,31.5825
4,20718173,2014-01-01,Armed Rebel (South Sudan),"Rebel,Dissident",South Sudan,Use unconventional violence,180,-9,Military (South Sudan),"Military,Government",South Sudan,32495113,1,The Australian,Juba,NaN,Central Equatoria State,South Sudan,4.85165,31.5825


In [4]:
len(df_)

879393

In [5]:
# Source and Target Sectors are kind of redundant with Source and Target Names
cols = ['Source Name', 'Source Country', 'Event Text', 'Target Name', 'Target Country']
df = df_.loc[:, cols]

# the category label space is massive, this reduces it
df['Source Name'].replace(to_replace='\s\(.+', value='', inplace=True, regex=True)
df['Target Name'].replace(to_replace='\s\(.+', value='', inplace=True, regex=True) 

# NaN values are when source/target don't have a country
# df[df.isnull().any(axis=1)]
df.fillna('NONE', inplace=True)

n_samples = 300000
df = df[:n_samples]

df.head()

,Source Name,Source Country,Event Text,Target Name,Target Country
0,Police,Australia,"Arrest, detain, or charge with legal action",Men,Australia
1,Police,Australia,"Arrest, detain, or charge with legal action",Children,Australia
2,Government Official,Democratic Republic of Congo,Make statement,Attacker,Democratic Republic of Congo
3,Military,South Sudan,Use conventional military force,Armed Rebel,South Sudan
4,Armed Rebel,South Sudan,Use unconventional violence,Military,South Sudan


In [6]:
cat_map = {}
for col in cols:
    cat = df.loc[:,col].astype('category')
    cat_map[col] = cat.cat.categories
    df.loc[:,col] = cat.cat.codes

In [7]:
cat_map['Source Country'][df['Source Country'][0]]

'Australia'

In [8]:
data = df.as_matrix()
data

array([[5477,   10,   39, 3618,    9],
       [5477,   10,   39, 1103,    9],
       [2493,   51,  169,  666,   50],
       ..., 
       [ 210,   67,   94, 4559,   30],
       [5477,   31,   94,  167,   66],
       [7303,  217,  236,  390,  220]], dtype=int16)

In [9]:
Y = data[1:]
Y.shape

(299999, 5)

In [10]:
data = data[:-1]
data.shape

(299999, 5)

In [11]:
vocab_size = np.max(data) + 1
vocab_size

7458

In [12]:
# check how many GB of memory will be needed
# using bool (8 bits)
print(Y.shape[0] * Y.shape[1] * (Y.max() + 1) * 8 * 1.25e-10)

y_onehot = np.zeros((Y.shape[0], Y.shape[1], Y.max() + 1), dtype=np.bool)
layer_idx = np.arange(Y.shape[0]).reshape(Y.shape[0], 1)
component_idx = np.tile(np.arange(Y.shape[1]), (Y.shape[0], 1))
y_onehot[layer_idx, component_idx, Y] = 1

11.18696271


In [13]:
from keras.layers.embeddings import Embedding
from keras.layers.core import Dropout, Dense
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed

embed_size = 50
batch_size = 256
epochs = 100

model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_length=data.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(embed_size, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(embed_size, return_sequences=True))
model.add(Dropout(0.3))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, y_onehot, batch_size=batch_size, nb_epoch=epochs, validation_split=0.05)

Train on 284999 samples, validate on 15000 samples
Epoch 1/100
284999/284999 [==============================] - 105s - loss: 5.7698 - acc: 0.0637 - val_loss: 5.3196 - val_acc: 0.0997
Epoch 2/100
284999/284999 [==============================] - 105s - loss: 5.1970 - acc: 0.0945 - val_loss: 5.1333 - val_acc: 0.1071
Epoch 3/100
284999/284999 [==============================] - 105s - loss: 5.0920 - acc: 0.0995 - val_loss: 5.0296 - val_acc: 0.1141
Epoch 4/100
284999/284999 [==============================] - 105s - loss: 5.0017 - acc: 0.1070 - val_loss: 4.9179 - val_acc: 0.1288
Epoch 5/100
284999/284999 [==============================] - 105s - loss: 4.9005 - acc: 0.1224 - val_loss: 4.8045 - val_acc: 0.1414
Epoch 6/100
284999/284999 [==============================] - 105s - loss: 4.7777 - acc: 0.1357 - val_loss: 4.6734 - val_acc: 0.1581
Epoch 7/100
284999/284999 [==============================] - 105s - loss: 4.6456 - acc: 0.1526 - val_loss: 4.5741 - val_acc: 0.1757
Epoch 8/100
284999/284999

Using TensorFlow backend.


In [14]:
model.save_weights('icews_weights.h5', overwrite=True)

In [15]:
def render(arr):
    parts = []
    for idx, col in enumerate(cols):
        parts.append(cat_map[col][int(arr[idx])])
    return parts

In [16]:
def ramble(seed, n):
    rambling = [seed]
    for _ in range(n):
        probs = model.predict(seed.T)[0]
        seed = []
        # random draw based on probs
        cs = np.cumsum(probs, axis=1)
        for row in cs:          
            idx = row.searchsorted(np.random.random() * row[-1], 'right')
            seed.append(idx)
        rambling.append(render(seed))
        seed = np.array(seed).reshape(5,1)
    return rambling

In [17]:
seed = np.random.randint(0, 20, (5,1))
for source_name, source_country, event_text, target_name, target_country in ramble(seed, 10):
    print('{} ({}) -> {} -> {} ({})'.format(source_name, source_country, event_text, target_name, target_country))

[2] ([17]) -> [7] -> [13] ([5])
A.B. Bardhan (Democratic Republic of Congo) -> Consult -> Greece (Venezuela)
France (United States) -> Consult -> Morocco (Ukraine)
Shrien Dewani (France) -> Host a visit -> Russia (NONE)
South Africa (South Africa) -> Host a visit -> Militant (Russian Federation)
Court Judge (India) -> Consult -> Citizen (Brazil)
Bangladesh Communist Party (India) -> Consult -> Citizen (India)
Farm Worker (India) -> Make statement -> Bharatiya Janata (India)
Mexico (India) -> Bring lawsuit against -> Indonesia (India)
Sudan People's Liberation Army (United States) -> Express intent to meet or negotiate -> United States (United Kingdom)
Military (Algeria) -> Express intent to meet or negotiate -> United States (Ukraine)
